<a href="https://colab.research.google.com/github/Oussamayousre/DLL/blob/version_1/Video_Generator_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras import datasets ,layers,models
import matplotlib.pyplot as plt
from keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications import *
import dlib
from keras.callbacks import ReduceLROnPlateau
import os 
import cv2
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.utils import Sequence
# i used a keras model , so i have to use data_utils.Sequence
from keras.utils import data_utils


In [28]:
detector = dlib.get_frontal_face_detector()

In [ ]:
!wget http://dlib.net/files/mmod_human_face_detector.dat.bz2


--2021-08-16 21:03:00--  http://dlib.net/files/mmod_human_face_detector.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694709 (678K)
Saving to: ‘mmod_human_face_detector.dat.bz2.1’

mmod_human_face_det 100%[===================>] 678.43K  1.60MB/s    in 0.4s    

2021-08-16 21:03:00 (1.60 MB/s) - ‘mmod_human_face_detector.dat.bz2.1’ saved [694709/694709]



In [ ]:
!bunzip2 /content/mmod_human_face_detector.dat.bz2

bunzip2: Output file /content/mmod_human_face_detector.dat already exists.


In [ ]:
cnn_face_detector = dlib.cnn_face_detection_model_v1('/content/mmod_human_face_detector.dat')


In [35]:
#next step is to collect all the videos in one folder and take the first character of their name which is the label 
import numpy as np
import keras
#path to video's folder
path = '/content/gdrive/MyDrive/variables/Fold2_part2'
#
train_ID = os.listdir(path)[1:]
class VideoFrameGenerator(data_utils.Sequence):
    'Generates data for Keras'
    def __init__(self,train_ID, batch_size=1, dim=(64,64), shuffle=True ):
        'Initialization'
        self.batch_size = batch_size
        self.train_ID = train_ID   
        self.shuffle = shuffle
        self.dim = dim
        self.on_epoch_end()
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.train_ID) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.train_ID[k] for k in indexes]
        #list_IDs_temp = [20]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        X =  np.array([X])[0]
        y =  np.array([y])
        y = y.reshape(y.shape[1])
        randomize = np.arange(y.shape[0])
        np.random.shuffle(randomize)
        X = X[randomize]
        y = y[randomize]
        return X , y 

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.train_ID))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):

        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        #X = np.empty((self.batch_size, *self.dim, self.n_channels))
        #y = np.empty((self.batch_size), dtype=int)
        arr_1 = os.listdir(f'/content/gdrive/MyDrive/variables/Fold2_part2/{list_IDs_temp[0]}')
        path_1  = f'/content/gdrive/MyDrive/variables/Fold2_part2/{list_IDs_temp[0]}'
        isDirectory = os.path.isdir(path_1)
        if isDirectory : 
          label_train  = []
          Frame_train = [] 
          # Generate data
          for path_3 in arr_1 :  
            if path_3 != '.ipynb_checkpoints' : 
                       
              label = int(os.path.splitext(path_3)[0])
              full_path_2 = f"/content/gdrive/MyDrive/variables/Fold2_part2/{list_IDs_temp[0]}/{path_3}"
              vs = cv2.VideoCapture(full_path_2)
              nb_frame = 0
              while True :
                ret, image = vs.read() 
                if ret == False  or nb_frame == 5000 : 
                    break
                #rects = detector(image,0)
                rects = detector(image, 0)
                for face in rects:                          
                  x1 = face.left() # left point
                  y1 = face.top() # top point
                  x2 = face.right() # right point
                  y2 = face.bottom() # bottom point
                  img = image[y1:y2,x1:x2]              
                  if img.size != 0 :
                    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(image, self.dim)
                    #image = image.img_to_array(image)
                    label_train.append(label)
                    Frame_train.append(img)
                    nb_frame += 1
              vs.release()
              cv2.destroyAllWindows()
        
        return Frame_train, label_train
     

In [36]:
path = '/content/gdrive/MyDrive/variables/Fold2_part2'
train_ID = os.listdir(path)[1:]

# Parameters
params = {'dim': (64,64),
          'batch_size': 1,
          'shuffle': True}
training_generator = VideoFrameGenerator(train_ID, **params)
validation_generator = VideoFrameGenerator([train_ID[0]], **params)
 

In [31]:
x ,y = training_generator.__getitem__(0)

In [32]:
x.shape

(30, 64, 64, 3)

In [37]:
vgg16 = VGG16( include_top=False, input_shape = (64,64, 3) , weights = 'imagenet' )


In [38]:
model = Sequential()

for layer in vgg16.layers[:-1]: # this is where I changed your code
    model.add(layer)    
# Freeze the layers 
#for layer in model.layers:
#    layer.trainable = False

In [39]:
model.add(Flatten())
model.add(Dense(11, activation='softmax'))

In [40]:
model.build((0,64,64,3))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 16, 16, 256)      

In [41]:

opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer = opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# try to capture face then train the model on faces directly
history = model.fit(training_generator, validation_data=validation_generator, epochs = 10 )

In [ ]:
model.save('/content/gdrive/MyDrive/variables')

In [ ]:
vs = cv2.VideoCapture("/content/gdrive/MyDrive/variables/dataset/0/0.mp4")
Frame_train = np.array([]).reshape(0,128)
label_train  = np.array([]).reshape(0,1)
COUNTER = 0 
while True : 
    ret, image = vs.read()
    if ret == False  : 
        break      
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = cv2.resize(image, (64,64))
    img = img.reshape(1,64,64,3)
    pred = np.argmax(model.predict(img))
    print(pred)
  #  cv2_imshow(frame)
 #   key = cv2.waitKey(1) & 0xFF
  #  if key == ord("q"):
       #     break

cv2.destroyAllWindows()